In [92]:
# Do all imports and installs here - Done
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
import pandas as pd
import re
import configparser
import os

In [93]:
# Parse configurations - Done
config = configparser.ConfigParser()
config.read('etl.cfg')

['etl.cfg']

#### Path 
input_data_source = '.'

output_processed_data = './storage'

#

i94immi_data_source = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'

worldtempe_data_source = '../../data2/GlobalLandTemperaturesByCity.csv'

citydemo_data_source = './us-cities-demographics.csv'

airport_data_source = './airport-codes_csv.csv'

#

i94_immi_splited_dir = './storage/.sas7bdat'

world_tempe_splited_dir = './storage/.csv'

In [114]:
# Create spark session
spark = SparkSession.builder\
            .config("spark.jars.repositories", "https://repos.spark-packages.org/")\
            .config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11")\
            .enableHiveSupport()\
            .getOrCreate()

In [ ]:
# df = spark.read.options(header='True',inferSchema='True',delimiter=',').csv(worldtempe_data_source)
# df = spark.read.options(header='True',inferSchema='True',delimiter=';').csv(citydemo_data_source)
# df = spark.read.options(header='True',inferSchema='True',delimiter=',').csv(airport_data_source)
# df = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [94]:
airport_dataset = './airport-codes_csv.csv'
airport_df = pd.read_csv(airport_dataset,sep=",")
pd.set_option('display.max_columns', 50)
airport_df.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


==========================================================================================================

==========================================================================================================

In [7]:
# Check airport distribution by country
airport_df[airport_df['iso_country'].isna()].count()

ident           247
type            247
name            247
elevation_ft    231
continent       247
iso_country       0
iso_region      247
municipality    145
gps_code         74
iata_code        31
local_code        0
coordinates     247
dtype: int64

In [8]:
# Check airport distribution by country
airport_df[airport_df['iso_country'].isna()].shape

(247, 12)

In [4]:
# Check country unique
airport_df['iso_country'].nunique()

243

In [5]:
airport_df = airport_df[airport_df['iso_country'] == 'US']
airport_df['iso_country'].nunique()

1

In [6]:
airport_df.isnull().sum()

ident               0
type                0
name                0
elevation_ft      239
continent       22756
iso_country         0
iso_region          0
municipality      102
gps_code         1773
iata_code       20738
local_code       1521
coordinates         0
dtype: int64

In [7]:
airport_df.duplicated().sum()

0

In [8]:
airport_df = airport_df.drop(['elevation_ft', 'continent', 'gps_code', 'local_code', 'coordinates'], axis=1)

In [9]:
airport_df.head()

,ident,type,name,iso_country,iso_region,municipality,iata_code
0,00A,heliport,Total Rf Heliport,US,US-PA,Bensalem,NaN
1,00AA,small_airport,Aero B Ranch Airport,US,US-KS,Leoti,NaN
2,00AK,small_airport,Lowell Field,US,US-AK,Anchor Point,NaN
3,00AL,small_airport,Epps Airpark,US,US-AL,Harvest,NaN
4,00AR,closed,Newport Hospital & Clinic Heliport,US,US-AR,Newport,NaN


In [19]:
airport_df = airport_df.dropna(subset=["iata_code"])

In [24]:
airport_df.shape

(2019, 7)

In [25]:
def convert_column_names(df):
    cols = df.columns
    column_name_changed = []

    for col in cols:
        new_column = col.lstrip().rstrip().lower().replace (" ", "_").replace ("-", "_") #strip beginning spaces, makes lowercase, add underscpre
        column_name_changed.append(new_column)

    df.columns = column_name_changed

In [27]:
convert_column_names(airport_df)
airport_df.columns

Index(['ident', 'type', 'name', 'iso_country', 'iso_region', 'municipality',
       'iata_code'],
      dtype='object')

In [ ]:
# Saving to CSV to staging
convert_column_names.to_csv("airports_df_clean.csv", index=False)